In [20]:
import pybaseball
from pybaseball import cache
from pybaseball import statcast
import pandas as pd

In [3]:
statcast_data = statcast('2025-03-28', '2025-03-29')

This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:05<00:00,  2.78s/it]


In [4]:
statcast_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
2799,FF,2025-03-29,93.8,-1.78,5.51,"Pagán, Emilio",682641,641941,field_out,hit_into_play,...,1,1.3,0.51,0.51,42.0,4.597651,-4.962009,23.996137,50.033604,37.403374
2897,FF,2025-03-29,95.3,-1.8,5.52,"Pagán, Emilio",527038,641941,field_out,hit_into_play,...,1,1.16,0.59,0.59,41.7,5.850287,3.003886,32.663956,38.065149,27.147809
3014,FF,2025-03-29,94.4,-1.78,5.59,"Pagán, Emilio",527038,641941,None,ball,...,1,1.16,0.62,0.62,42.2,<NA>,<NA>,<NA>,<NA>,<NA>
3143,FS,2025-03-29,84.0,-1.58,5.52,"Pagán, Emilio",527038,641941,None,ball,...,1,2.64,1.0,1.0,45.7,<NA>,<NA>,<NA>,<NA>,<NA>
3277,FF,2025-03-29,93.3,-1.71,5.61,"Pagán, Emilio",527038,641941,None,swinging_strike,...,1,1.12,0.44,0.44,39.5,4.422645,3.885418,23.835304,24.165759,24.090179


In [13]:
# cols i want:
cols=['game_date', 'pitch_type', 'p_throws', 'stand', 'events', 'description', 'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z', 'player_name', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'effective_speed', 'release_spin_rate', 'release_extension', 'pitcher', 'spin_axis']

In [14]:
filt_data = statcast_data[cols]

In [15]:
# calculating (approx) VAA and HAA

import numpy as np

def calculate_vaa_haa(df):
    # horizontal (x), vertical (z), and distance to plate (y)
    dx = df['plate_x'] - df['release_pos_x']
    dz = df['plate_z'] - df['release_pos_z']
    dy = -df['release_pos_y']  # plate at y = 0

    df['VAA'] = np.degrees(np.arctan2(dz, np.abs(dy)))
    df['HAA'] = np.degrees(np.arctan2(dx, np.abs(dy)))

    return df

filt_data = calculate_vaa_haa(filt_data)
filt_data.describe()


/var/folders/h6/_rz_hgwd06q2j4rsml2mr9yr0000gn/T/ipykernel_83122/3118122403.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VAA'] = np.degrees(np.arctan2(dz, np.abs(dy)))
/var/folders/h6/_rz_hgwd06q2j4rsml2mr9yr0000gn/T/ipykernel_83122/3118122403.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HAA'] = np.degrees(np.arctan2(dx, np.abs(dy)))


,game_date,release_speed,release_pos_x,release_pos_y,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,effective_speed,release_spin_rate,release_extension,pitcher,spin_axis,VAA,HAA
count,6916,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6885.0,6883.0,6885.0,6885.0,6916.0,6885.0,6885.0,6885.0
mean,2025-03-28 15:16:33.175245,89.538141,-1.045949,54.114065,5.728649,-0.150144,0.600076,0.050731,2.313275,89.561572,2287.153522,6.385882,644113.063331,176.157008,-3.610221,1.161932
min,2025-03-28 00:00:00,55.3,-4.27,52.34,3.64,-1.88,-1.74,-2.98,-1.46,52.8,418.0,3.8,434378.0,1.0,-8.068986,-6.727209
25%,2025-03-28 00:00:00,85.4,-2.26,53.83,5.48,-0.93,0.11,-0.51,1.68,85.3,2148.0,6.1,622491.0,128.0,-4.313438,0.319016
50%,2025-03-29 00:00:00,90.3,-1.73,54.11,5.78,-0.29,0.61,0.04,2.32,90.5,2331.0,6.4,663559.0,204.0,-3.595623,1.674714
75%,2025-03-29 00:00:00,94.2,-0.67,54.41,6.01,0.55,1.24,0.62,2.98,94.2,2491.0,6.7,678821.0,225.0,-2.882813,2.586682
max,2025-03-29 00:00:00,102.8,4.65,56.71,7.31,2.12,1.96,3.8,6.37,104.5,3435.0,8.2,808967.0,360.0,0.669362,6.633709
std,NaN,5.740167,1.801798,0.448786,0.463881,0.911672,0.716934,0.836298,0.96937,5.893679,370.693313,0.44979,61751.233032,72.091796,1.097972,2.090709


In [16]:
filt_data.columns

Index(['game_date', 'pitch_type', 'p_throws', 'stand', 'events', 'description',
       'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z',
       'player_name', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
       'effective_speed', 'release_spin_rate', 'release_extension', 'pitcher',
       'spin_axis', 'VAA', 'HAA'],
      dtype='object')

In [17]:
# define events of interest: 
swinging_strikes = {
    'swinging_strike',
    'swinging_strike_blocked'
}

swings = {
    'foul', 'foul_tip', 'swinging_strike',
    'swinging_strike_blocked', 'hit_into_play'
}

called_strikes = {'called_strike'}
balls = {'ball', 'blocked_ball'}


In [18]:
# Aggregating predictive stats per pitcher:
df=filt_data.copy()
df['season'] = df['game_date'].dt.year

# binary cols:
df['is_swing'] = df['description'].isin(swings)
df['is_whiff'] = df['description'].isin(swinging_strikes)
df['is_called_strike'] = df['description'].isin(called_strikes)
df['is_ball'] = df['description'].isin(balls)

# aggregate to pitcher-pitch_type-season level:
agg = (
    df.groupby(['pitcher', 'pitch_type', 'season'])
      .agg(
          pitches=('description', 'count'),
          swings=('is_swing', 'sum'),
          whiffs=('is_whiff', 'sum'),
          called_strikes=('is_called_strike', 'sum'),
          balls=('is_ball', 'sum')
      )
      .reset_index()
)

agg['whiff_pct'] = agg['whiffs'] / agg['swings']
agg['csw_pct'] = (agg['whiffs'] + agg['called_strikes']) / agg['pitches']
agg.head()



,pitcher,pitch_type,season,pitches,swings,whiffs,called_strikes,balls,whiff_pct,csw_pct
0,434378,CH,2025,9,2,1,1,6,0.500000,0.222222
1,434378,CU,2025,10,3,1,4,3,0.333333,0.500000
2,434378,FF,2025,43,22,4,7,14,0.181818,0.255814
3,434378,SL,2025,10,7,2,1,2,0.285714,0.300000
4,434378,ST,2025,11,4,0,3,4,0.000000,0.272727


In [19]:
# aggregating descriptors to season level too
desc = (
    df.groupby(['pitcher', 'pitch_type', 'season'])
      .agg(
          n_pitches=('description', 'count'),
          velo=('release_speed', 'mean'),
          pfx_x=('pfx_x', 'mean'),
          pfx_z=('pfx_z', 'mean'),
          VAA=('VAA', 'mean'),
          HAA=('HAA', 'mean'),
          ext=('release_extension', 'mean'),
          spin=('release_spin_rate', 'mean'),
          rel_x=('release_pos_x', 'mean'),
          rel_z=('release_pos_z', 'mean'),
          plate_x=('plate_x', 'mean'),
          plate_z=('plate_z', 'mean'),
      )
      .reset_index()
)

# usage
total = desc.groupby(['pitcher', 'season'])['n_pitches'].transform('sum')
desc['usage'] = desc['n_pitches'] / total
desc.head()

,pitcher,pitch_type,season,n_pitches,velo,pfx_x,pfx_z,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage
0,434378,CH,2025,9,85.644444,-0.888889,0.731111,-5.925198,1.775221,6.388889,1525.222222,-1.893333,6.778889,-0.215556,1.161111,0.108434
1,434378,CU,2025,10,79.7,0.52,-0.816,-4.727961,1.769858,6.23,2720.9,-1.71,6.887,-0.033,2.398,0.120482
2,434378,FF,2025,43,94.1,-0.609535,1.480465,-4.203331,1.682875,6.202326,2459.069767,-1.586744,6.95814,0.008837,2.966744,0.518072
3,434378,SL,2025,10,88.18,0.387,0.458,-5.193331,2.307576,6.27,2451.4,-1.677,6.872,0.508,1.943,0.120482
4,434378,ST,2025,11,78.781818,1.045455,-0.449091,-5.015678,2.131764,6.154545,2642.545455,-2.001818,6.823636,0.020909,2.053636,0.132530


In [21]:
# Merging with stats/descriptors:
final = pd.merge(agg, desc, on=['pitcher', 'pitch_type', 'season'])
final.head()

,pitcher,pitch_type,season,pitches,swings,whiffs,called_strikes,balls,whiff_pct,csw_pct,...,pfx_z,VAA,HAA,ext,spin,rel_x,rel_z,plate_x,plate_z,usage
0,434378,CH,2025,9,2,1,1,6,0.500000,0.222222,...,0.731111,-5.925198,1.775221,6.388889,1525.222222,-1.893333,6.778889,-0.215556,1.161111,0.108434
1,434378,CU,2025,10,3,1,4,3,0.333333,0.500000,...,-0.816,-4.727961,1.769858,6.23,2720.9,-1.71,6.887,-0.033,2.398,0.120482
2,434378,FF,2025,43,22,4,7,14,0.181818,0.255814,...,1.480465,-4.203331,1.682875,6.202326,2459.069767,-1.586744,6.95814,0.008837,2.966744,0.518072
3,434378,SL,2025,10,7,2,1,2,0.285714,0.300000,...,0.458,-5.193331,2.307576,6.27,2451.4,-1.677,6.872,0.508,1.943,0.120482
4,434378,ST,2025,11,4,0,3,4,0.000000,0.272727,...,-0.449091,-5.015678,2.131764,6.154545,2642.545455,-2.001818,6.823636,0.020909,2.053636,0.132530


In [22]:
final.to_parquet('initial_short_pitches_description_whiff_csw_stats.parquet')